In [2]:
import pandas as pd
import os
from google.cloud import bigquery
import json
from google.oauth2 import service_account
import db_dtypes 

In [3]:
# Obtain the key from the environment variable
service_account_key =os.getenv('GCP_ACCESS_KEY')
if not service_account_key:
    raise ValueError("The GCP_SERVICE_ACCOUNT_KEY environment variable is not set")

key_data = json.loads(service_account_key)

credentials = service_account.Credentials.from_service_account_info(
    key_data,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)


In [4]:
service_acc_key=os.getenv('GCP_ACCESS_KEY')

In [5]:
project_id = os.getenv('GOOGLE_PROJECT_MAIN_FP')

In [9]:
client = bigquery.Client(credentials=credentials,project=project_id)

In [10]:
sql = """
SELECT * FROM `bigquery-public-data.san_francisco.bikeshare_trips` LIMIT 10
 """

In [11]:
# import pandas_gbq
# df = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials, progress_bar_type=None)

In [12]:

df = client.query(sql).to_dataframe()

In [13]:
df.head()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


In [18]:
df.shape

(151245, 3)

In [17]:
df['city_name'].value_counts()

city_name
Atlanta             18942
Washington          18942
Chicago             18932
Ciudad de México    18932
Seattle             18912
San Francisco       18912
New York            18850
Los Angeles         18823
Name: count, dtype: int64

#### BigQuery Model Creations

In [14]:
table_name = "'bigquery-public-data.san_francisco.bikeshare_trips'"

In [18]:
query_papams = [
    bigquery.ScalarQueryParameter("subscriber_type", "STRING", "Customer"),
]
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_papams

sql = """ SELECT * FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
WHERE subscriber_type = @subscriber_type
  """

query_job = client.query(sql, job_config=job_config)
query_job.result()
df_result = query_job.to_dataframe()
print(" query results loaded to the table")

 query results loaded to the table


In [19]:
df_result

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer
...,...,...,...,...,...,...,...,...,...,...,...
136804,1323441,102,2016-08-19 16:56:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-19 16:58:00+00:00,Cyril Magnin St at Ellis St,91,473,55104,Customer
136805,1327216,7913,2016-08-23 14:02:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-23 16:14:00+00:00,Cyril Magnin St at Ellis St,91,377,90049,Customer
136806,1320788,564,2016-08-18 06:41:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-18 06:50:00+00:00,Cyril Magnin St at Ellis St,91,295,20025,Customer
136807,1320548,7000,2016-08-17 18:26:00+00:00,Cyril Magnin St at Ellis St,91,2016-08-17 20:22:00+00:00,Cyril Magnin St at Ellis St,91,226,nil,Customer


In [ ]:
sql = """
SELECT * FROM `bigquery-public-data.covid19_geotab_mobility_impact.city_congestion` 
 """
model_creation = client.query(sql).to_dataframe()